PART B : Stoichometry and reaction balancing
1: Reaction Balance Function
    In this part, we will create a function that can balance a reaction equation and give the stoichometric coefficient for each molecule that balance the reaction.

In [ ]:
import re
import numpy as np
from scipy.linalg import null_space

def parse_formula(formula):
    # The objective of this function is to take a chemical formula
    # and to transform it into a dictionary of elements and their 
    # atom counts
    pattern = r'([A-Z][a-z]?)(\d*)'
    # pattern explanation: [A-Z] matches an uppercase letter and
    # [a-z]? matches an optional lowercase letter, ex: "H" or "He"
    # together they match an element symbol
    # (\d*) matches the number that follows the element 
    elements = re.findall(pattern, formula)
    # elements will be a list of tuples (element, count) throught the
    # findall function
    counts = {}
    for (element, count) in elements:
        counts[element] = counts.get(element, 0) + int(count or 1)
    #This part will count the number of atoms for each element, for 
    # each (element, count), count.get(element, 0) will return the stocked 
    # value, otherwise it is 0, int(count or 1) will convert count to integer,
    # if count is empty, it will return 1
    return counts

def Balance_reaction(reactants, products):
    # This function will balance a chemical reaction given the
    # list of reactants and products, and return the stoichiometric
    # coefficients for each molecule in the reaction
    all_elements = set()
    for formula in reactants + products:
        all_elements.update(parse_formula(formula).keys())
    all_elements = sorted(all_elements)
    # This part will put all the elements of the reactants and products
    # into a set to avoid duplicates and then we sort them
    # Example: for H2 + O2 -> H2O, all_elements will be ['H', 'O']


    n = len(all_elements)
    m = len(reactants) + len(products)
    matrix = np.zeros((n, m))
    # This part build the matrix for the system of equations where:
    # n is the number of unique elements
    # m is the number of molecules (reactants + products)
    # we initialize a nxm matrix of zeros

    for i, element in enumerate(all_elements):
        for j, formula in enumerate(reactants):
            matrix[i, j] = parse_formula(formula).get(element, 0)
        for j, formula in enumerate(products):
            matrix[i, j + len(reactants)] = -parse_formula(formula).get(element, 0)
    # This part will fill the matrix with the atom counts, 
    # for reactants we put positive counts and for products negative counts
    # The final equation system will be Ax = 0, where A is the matrix and x is the
    # stoichometric coefficients we want to find

    # Use scipy's null_space to get a solution vector
    ns = null_space(matrix)
    if ns.size == 0:
        raise ValueError("No solution found for balancing the reaction.")
    coeffs = ns[:, 0]
    coeffs = coeffs / np.min(np.abs(coeffs[np.nonzero(coeffs)]))
    coeffs = np.round(coeffs).astype(int)
    return coeffs.tolist()
    # We use scipy (scipy.linalg) to directly calculate the null space
    # (which are the solution of Ax=0)
    # If ns is empty, this means that there is no trivial solutions and the 
    # reaction cannot be balanced
    # Then the vector is normalized to have the smallest coefficient is equal 
    # to 1, we then round the real values to integers and return the list of 
    # the coefficients

# Example usage:

# Example 1:
reactants = ["H2", "O2"]
products = ["H2O"]
coefficients = Balance_reaction(reactants, products)
print("Coefficients:", coefficients)  
# Output should be [2, 1, 2] for 2H2 + 1O2 -> 2H2O

# Example 2:
reactants2 = ["C3H8", "O2"]
products2 = ["CO2", "H2O"]
coefficients2 = Balance_reaction(reactants2, products2)
print("Coefficients 2:", coefficients2)
# Output should be [1, 5, 3, 4]

# Example 3:
reactants3 = ["C2H5OH", "O2"]
products3 = ["CO2", "H2O"]
coefficients3 = Balance_reaction(reactants3, products3)
print("Coefficients 3:", coefficients3)
# Output should be [1, 3, 2, 3]

Coefficients: [2, 1, 2]
Coefficients 2: [1, 5, 3, 4]
Coefficients 3: [1, 3, 2, 3]


2: Mass conservation 
    In this part, we will use the molecular mass calculator from part A
    to compute the total mass of the reactants and products with the coefficients.

In [16]:
path = "periodic_table.csv"
import pandas as pd
#Load the "periodic_table.csv" file and create a dictionary mapping element symbols to their atomic masses.
data = pd.read_csv(path)
atomic_masses = dict(zip(data['Symbol'], data['AtomicMass']))

def calculate_molecular_mass(formula):
    #This function calculates the molecular mass of a chemical formula without parentheses. It parses the formula character by character, identifying elements and their counts, and sums their contributions to the total molecular mass.
    #initialize total mass and index
    total_mass = 0.0
    i = 0
    #loops over the chemical formula
    while i < len(formula):
        #parsing of the element symbol
        element = formula[i]
        i += 1
        if i < len(formula) and formula[i].islower():
            element += formula[i]
            i += 1
        #parsing the count of atoms
        count_str = ''
        while i < len(formula) and formula[i].isdigit():
            count_str += formula[i]
            i += 1
        count = int(count_str) if count_str else 1
        #adding the mass contribution of the element to the total mass
        if element in atomic_masses:
            total_mass += atomic_masses[element] * count
        else:
            raise ValueError(f"Element {element} not found in atomic masses.")
    
    return total_mass

# This is the function that calculates the molecular mass of a given molecule from part A

def total_mass(molecules_dict):
    total = 0.0
    for formula, coeff in molecules_dict.items():
        total += calculate_molecular_mass(formula) * coeff
    return total
# This function will loop over the dictionary, for each molecule, it will calculate its molecular mass
# using the function from part A, multiply it by its coefficient and add it to the total

def check_mass_conservation(reactants, products, reactants_coeff, products_coeff, tolerance=1e-6):
    # We construct a dictionary for the reactants and products with their coefficients {formula: coeff}
    reactants_dict = {reactants[i]: reactants_coeff[i] for i in range(len(reactants))}
    products_dict = {products[i]: products_coeff[i] for i in range(len(products))}

    # We calculate the total mass of the reactants and products
    react_mass = total_mass(reactants_dict)
    prod_mass = total_mass(products_dict)

    # We check if the masses are equal (within a small tolerance to account for floating-point precision)
    if abs(react_mass - prod_mass) < tolerance:
        return print(f"The mass is conserved: Reactants mass = {react_mass} u, Products mass = {prod_mass} u")
        
    else:
        return print(f"The mass is NOT conserved: Reactants mass = {react_mass} u, Products mass = {prod_mass} u")
        

# Example usage for mass conservation check:
# Using the first example reaction
reactants = ["H2", "O2"]
products = ["H2O"]
reactants_coeff = [2, 1]
products_coeff = [2]
check_mass_conservation(reactants, products, reactants_coeff, products_coeff, tolerance=1e-6)

# Example usage for mass conservation check where the mass is not conserved:
# Using the second example reaction with incorrect coefficients
reactants2 = ["C3H8", "O2"] 
products2 = ["CO2", "H2O"]
reactants_coeff2 = [1, 4]  # Incorrect coefficient for O2
products_coeff2 = [3, 4]
check_mass_conservation(reactants2, products2, reactants_coeff2, products_coeff2, tolerance=1e-6)


The mass is conserved: Reactants mass = 36.03 u, Products mass = 36.03 u
The mass is NOT conserved: Reactants mass = 172.089 u, Products mass = 204.087 u
